In [3]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, get_feature_names

In [5]:
data = pd.read_csv("F:/BI/第五周/核心板/L5/deepfm/movielens_sample.txt")
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ['rating']

In [6]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

In [7]:
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [8]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [9]:
# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=50, verbose=True, validation_split=0.2, )

Epoch 1/50


F:\Anaconda_download\anaconda_0\lib\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1/1 [==============================] - 0s 312ms/step - loss: 14.0155 - mse: 14.0155 - val_loss: 13.3590 - val_mse: 13.3590
Epoch 2/50
1/1 [==============================] - 0s 23ms/step - loss: 13.8776 - mse: 13.8776 - val_loss: 13.2429 - val_mse: 13.2429
Epoch 3/50
1/1 [==============================] - 0s 22ms/step - loss: 13.7298 - mse: 13.7298 - val_loss: 13.1182 - val_mse: 13.1182
Epoch 4/50
1/1 [==============================] - 0s 25ms/step - loss: 13.5720 - mse: 13.5720 - val_loss: 12.9858 - val_mse: 12.9858
Epoch 5/50
1/1 [==============================] - 0s 22ms/step - loss: 13.4044 - mse: 13.4044 - val_loss: 12.8449 - val_mse: 12.8449
Epoch 6/50
1/1 [==============================] - 0s 22ms/step - loss: 13.2265 - mse: 13.2265 - val_loss: 12.6951 - val_mse: 12.6951
Epoch 7/50
1/1 [==============================] - 0s 25ms/step - loss: 13.0375 - mse: 13.0375 - val_loss: 12.5357 - val_mse: 12.5357
Epoch 8/50
1/1 [==============================] - 0s 24ms/step - loss: 12.8363 

In [10]:
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
print("MSE", mse)

MSE 1.8667
